In [1]:
%load_ext autoreload
%autoreload 2

import os
import xarray as xr
import dask
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

from aurora_benchmark.utils import xr_to_netcdf, update_running_statistics, reduce_statistics

dask.config.set(scheduler='threads')

In [2]:
p = "../toy_data/era5-1d-360x180/msl-2021-2022-1d-360x180.nc"
surf_vars = ["2t", "msl", "10u"]
atmospheric_vars = ["t", "q", "u"]
static_vars = ["z", "lsm", "slt"]

In [7]:
# Function arguments
toy = True
print(f"TOY: {toy}")

if toy:
    era5_surface_paths = [
        p, p, p
    ]
    era5_atmospheric_paths = [
        p, p, p
    ]
    era5_static_paths = [
        p, p, p
    ]

    interest_variables = atmospheric_vars + surf_vars
    interest_levels = [1000, 700, 250]

    forecast_dir = "data/era5_wb2_forecasts/2021-2022-6h-1d-6w-1440x721_original_variables/"
    eval_dir = "data/era5_wb2_eval/2021-2022-6h-1d-6w-1440x721_original_variables"
    
    # Load the data into a single dataset with the same coords but multiple variables
    surface_ds = xr.merge([
        xr.open_dataset(path, engine="netcdf4")\
            .rename({"msl": svar})\
            .drop_vars("time_bnds")
        for path, svar in zip(era5_surface_paths, surf_vars)
    ])

    atmospheric_ds = xr.merge([
        xr.open_dataset(path, engine="netcdf4") \
            .rename({"msl": svar}) \
            .expand_dims({"level": interest_levels}) \
            .drop_vars("time_bnds")
        for path, svar in zip(era5_atmospheric_paths, atmospheric_vars)
    ])

    static_ds = xr.merge([
        xr.open_dataset(path, engine="netcdf4").rename({"msl": svar}).isel(time=0).drop_vars("time_bnds")
        for path, svar in zip(era5_atmospheric_paths, atmospheric_vars)
    ])

    # rename coord lat to latitude and lon to longitude
    surface_ds = surface_ds.rename({"lat": "latitude", "lon": "longitude"})
    atmospheric_ds = atmospheric_ds.rename({"lat": "latitude", "lon": "longitude"})
    static_ds = static_ds.rename({"lat": "latitude", "lon": "longitude"})
   
else: 
    era5_surface_paths = [
    "/projects/prjs0981/ewalt/aurora_benchmark/data/era5_wb2/2021-2022-6h-1440x721/10v_2021-2022-6h-1440x721.nc",
    "/projects/prjs0981/ewalt/aurora_benchmark/data/era5_wb2/2021-2022-6h-1440x721/2t_2021-2022-6h-1440x721.nc",
    #  - data/era5_wb2/2021-2022-6h-1440x721/tp_2021-2022-6h-1440x721.nc # original only
    "/projects/prjs0981/ewalt/aurora_benchmark/data/era5_wb2/2021-2022-6h-1440x721/10u_2021-2022-6h-1440x721.nc",
    "/projects/prjs0981/ewalt/aurora_benchmark/data/era5_wb2/2021-2022-6h-1440x721/msl_2021-2022-6h-1440x721.nc",
    #  - data/era5_wb2/2021-2022-6h-1440x721/sst_2021-2022-6h-1440x721.nc # original only
    ]
    era5_atmospheric_paths = [
    "/projects/prjs0981/ewalt/aurora_benchmark/data/era5_wb2/2021-2022-6h-1440x721/q_2021-2022-6h-1440x721.nc",
    "/projects/prjs0981/ewalt/aurora_benchmark/data/era5_wb2/2021-2022-6h-1440x721/t_2021-2022-6h-1440x721.nc",
    "/projects/prjs0981/ewalt/aurora_benchmark/data/era5_wb2/2021-2022-6h-1440x721/u_2021-2022-6h-1440x721.nc",
    "/projects/prjs0981/ewalt/aurora_benchmark/data/era5_wb2/2021-2022-6h-1440x721/v_2021-2022-6h-1440x721.nc",
    "/projects/prjs0981/ewalt/aurora_benchmark/data/era5_wb2/2021-2022-6h-1440x721/z_2021-2022-6h-1440x721.nc",
    ]
    era5_static_paths = [
    "/projects/prjs0981/ewalt/aurora_benchmark/data/era5_wb2/2021-2022-6h-1440x721/lsm_static-1440x721.nc",
    "/projects/prjs0981/ewalt/aurora_benchmark/data/era5_wb2/2021-2022-6h-1440x721/z_static-1440x721.nc",
    "/projects/prjs0981/ewalt/aurora_benchmark/data/era5_wb2/2021-2022-6h-1440x721/slt_static-1440x721.nc",
    ]

    interest_variables = atmospheric_vars + surf_vars
    interest_levels = [1000, 700, 250]

    forecast_dir = "/projects/prjs0981/ewalt/aurora_benchmark/data/era5_wb2_forecasts/2021-2022-6h-1d-6w-1440x721_original_variables/"
    eval_dir = "data/era5_wb2_eval/2021-2022-6h-1d-6w-1440x721_original_variables"
    
    # Load the data into a single dataset with the same coords but multiple variables
    surface_ds = xr.merge([
        xr.open_dataset(p, engine="netcdf4", chunks={"time": 50, "latitude": 721, "longitude": 1440})
        for p in era5_surface_paths
    ])

    atmospheric_ds = xr.merge([
        xr.open_dataset(p, engine="netcdf4", chunks={"time": 50, "latitude": 721, "longitude": 1440, "level": 1})
        for p in era5_atmospheric_paths
    ])

    static_ds = xr.merge([
        xr.open_dataset(p, engine="netcdf4", chunks={"latitude": 721, "longitude": 1440})
        for p in era5_static_paths
    ])


assert os.path.exists(forecast_dir), f"Forecast directory {forecast_dir} does not exist"
os.makedirs(eval_dir, exist_ok=True)

surface_ds.dims, atmospheric_ds.dims, static_ds.dims

TOY: True


(FrozenMappingWarningOnValuesAccess({'time': 365, 'longitude': 360, 'latitude': 180}),
 FrozenMappingWarningOnValuesAccess({'level': 3, 'time': 365, 'longitude': 360, 'latitude': 180}),
 FrozenMappingWarningOnValuesAccess({'longitude': 360, 'latitude': 180}))

In [13]:
import xarray as xr

class RunningStatistics(object):
    def __init__(self):
        self._means = {}
        self.means_of_squared = {}
        self.num_samples = {}

    def update(self, sample_ds: xr.Dataset):
        for var in sample_ds.data_vars:
            if var not in self.num_samples:
                print(f"New variable {var}")
                self.num_samples[var] = 0
                self._means[var] = sample_ds[var]
                self.means_of_squared[var] = sample_ds[var]**2
                print(f"means[var] shape:", self._means[var].dims)
                print(f"means_of_squared[var] shape:", self.means_of_squared[var].dims)
            else:
                self._means[var] = (self._means[var] * self.num_samples[var] + sample_ds[var]) / (self.num_samples[var] + 1)
                self.means_of_squared[var] = (self.means_of_squared[var] * self.num_samples[var] + sample_ds[var]**2) / (self.num_samples[var] + 1)
            self.num_samples[var] += 1

    @property
    def stds(self):
        return xr.Dataset({var: (self.means_of_squared[var] - self._means[var]**2)**0.5 for var in self._means})
    
    @property
    def means(self):
        return xr.Dataset(self._means)

In [14]:
running_statistics = {
    "surface_vars": RunningStatistics(),
    "atmospheric_vars": RunningStatistics(),
}

# loop over files
for i, file in enumerate(os.scandir(forecast_dir)):
    # parse file info
    file_info = file.name.replace(".nc", "").split("_")
    variable_name = file_info[1]
    file_info = file_info[2].split("-")
    init_time = pd.Timestamp(file_info[0])
    base_frequency = file_info[1]
    eval_aggregation = file_info[2]
    eval_start = file_info[3]
    forecast_horizon = file_info[4]    

    # feedback
    print(f"Processing {file.name}")

    # load forecast
    pred_trajectory = xr.open_dataset(file.path, engine="netcdf4")
    assert pd.Timedelta((pred_trajectory.lead_time[1]-pred_trajectory.lead_time[0]).values) == pd.Timedelta(eval_aggregation)

    # load ERA5 gt from surface_ds and atmospheric_ds
    true_ds = atmospheric_ds if variable_name in atmospheric_ds.data_vars else surface_ds
    true_trajectory = true_ds[variable_name]\
            .sel(time=slice(init_time+pd.Timedelta(eval_start), init_time+pd.Timedelta(forecast_horizon)))

    # resample gt to eval_aggregation
    true_trajectory = true_trajectory.resample(time=pd.Timedelta(eval_aggregation), origin=init_time).mean()
    assert pd.Timedelta((true_trajectory.time[1]-true_trajectory.time[0]).values) == pd.Timedelta(eval_aggregation)
    
    # rename true time to lead time
    true_trajectory = true_trajectory.rename({"time": "lead_time"})
    true_trajectory["lead_time"] = true_trajectory["lead_time"] - np.datetime64(init_time)

    # shape
    sizes = pred_trajectory.sizes
    nlt = len(np.unique(pred_trajectory.lead_time.values))
    if variable_name in atmospheric_ds.data_vars:
        stat_key = "atmospheric_vars"
    else:
        stat_key = "surface_vars"
        
    # compute error
    signed_error_ds = (pred_trajectory - true_trajectory)
    running_statistics[stat_key].update(signed_error_ds)
    
    if i == 50: break

Processing forecast_u_20210228T090000-1d-1w-1w-3w-360x180.nc
New variable u
means[var] shape: ('lead_time', 'level', 'latitude', 'longitude')
means_of_squared[var] shape: ('lead_time', 'level', 'latitude', 'longitude')
Processing forecast_u_20210401T090000-1d-1w-1w-3w-360x180.nc
Processing forecast_msl_20210412T090000-1d-1w-1w-3w-360x180.nc
New variable msl
means[var] shape: ('lead_time', 'latitude', 'longitude')
means_of_squared[var] shape: ('lead_time', 'latitude', 'longitude')
Processing forecast_u_20210116T090000-1d-1w-1w-3w-360x180.nc
Processing forecast_t_20210420T090000-1d-1w-1w-3w-360x180.nc
New variable t
means[var] shape: ('lead_time', 'level', 'latitude', 'longitude')
means_of_squared[var] shape: ('lead_time', 'level', 'latitude', 'longitude')
Processing forecast_10u_20210322T090000-1d-1w-1w-3w-360x180.nc
New variable 10u
means[var] shape: ('lead_time', 'latitude', 'longitude')
means_of_squared[var] shape: ('lead_time', 'latitude', 'longitude')
Processing forecast_q_20210612

In [21]:
running_statistics["surface_vars"].means["10u"].values

array([], shape=(2, 180, 360, 0), dtype=float32)

In [23]:
signed_error_ds

<xarray.Dataset> Size: 2MB
Dimensions:    (latitude: 180, longitude: 360, level: 3, lead_time: 2)
Coordinates:
  * latitude   (latitude) float64 1kB 89.5 88.5 87.5 86.5 ... -87.5 -88.5 -89.5
  * longitude  (longitude) float32 1kB 0.0 1.0 2.0 3.0 ... 357.0 358.0 359.0
  * level      (level) int64 24B 1000 700 250
  * lead_time  (lead_time) timedelta64[ns] 16B 7 days 14 days
Data variables:
    q          (lead_time, level, latitude, longitude) float32 2MB -1.011e+05...

In [28]:
(pred_trajectory-true_trajectory)["q"] + (pred_trajectory-true_trajectory)["q"] 

<xarray.DataArray 'q' (lead_time: 2, level: 3, latitude: 180, longitude: 360)> Size: 2MB
array([[[[-202107.9 , -202105.27, -202102.56, ..., -202116.23,
          -202113.48, -202110.6 ],
         [-202155.95, -202148.22, -202140.39, ..., -202176.34,
          -202169.56, -202162.78],
         [-202201.83, -202189.33, -202176.66, ..., -202234.33,
          -202223.5 , -202212.73],
         ...,
         [-200041.78, -200046.1 , -200040.45, ..., -200007.58,
          -200015.64, -200024.08],
         [-200299.66, -200312.16, -200324.83, ..., -200273.25,
          -200281.9 , -200290.75],
         [-200563.8 , -200562.88, -200560.36, ..., -200564.11,
          -200563.67, -200563.22]],

        [[-202107.9 , -202105.27, -202102.56, ..., -202116.23,
          -202113.48, -202110.6 ],
         [-202155.95, -202148.22, -202140.39, ..., -202176.34,
          -202169.56, -202162.78],
         [-202201.83, -202189.33, -202176.66, ..., -202234.33,
          -202223.5 , -202212.73],
...
         [-202986.47, -202982.36, -202966.  , ..., -202972.  ,
          -202971.92, -202972.05],
         [-203068.53, -203083.75, -203098.92, ..., -203038.34,
          -203048.39, -203058.47],
         [-203027.28, -203026.47, -203023.89, ..., -203024.75,
          -203025.22, -203025.61]],

        [[-201411.05, -201411.16, -201411.34, ..., -201412.  ,
          -201411.64, -201411.28],
         [-200962.89, -200961.88, -200961.05, ..., -200967.39,
          -200965.67, -200964.14],
         [-200518.97, -200514.75, -200510.78, ..., -200532.47,
          -200527.61, -200523.14],
         ...,
         [-202986.47, -202982.36, -202966.  , ..., -202972.  ,
          -202971.92, -202972.05],
         [-203068.53, -203083.75, -203098.92, ..., -203038.34,
          -203048.39, -203058.47],
         [-203027.28, -203026.47, -203023.89, ..., -203024.75,
          -203025.22, -203025.61]]]], dtype=float32)
Coordinates:
  * latitude   (latitude) float64 1kB 89.5 88.5 87.5 86.5 ... -87.5 -88.5 -89.5
  * longitude  (longitude) float32 1kB 0.0 1.0 2.0 3.0 ... 357.0 358.0 359.0
  * level      (level) int64 24B 1000 700 250
  * lead_time  (lead_time) timedelta64[ns] 16B 7 days 14 days

In [33]:
running_statistics["surface_vars"].means_of_squared

{'msl': <xarray.DataArray 'msl' (lead_time: 2, latitude: 180, longitude: 360, time: 0)> Size: 0B
 array([], shape=(2, 180, 360, 0), dtype=float32)
 Coordinates:
   * latitude   (latitude) float64 1kB 89.5 88.5 87.5 86.5 ... -87.5 -88.5 -89.5
   * longitude  (longitude) float32 1kB 0.0 1.0 2.0 3.0 ... 357.0 358.0 359.0
   * lead_time  (lead_time) timedelta64[ns] 16B 7 days 14 days
   * time       (time) datetime64[ns] 0B ,
 '10u': <xarray.DataArray '10u' (lead_time: 2, latitude: 180, longitude: 360, time: 0)> Size: 0B
 array([], shape=(2, 180, 360, 0), dtype=float32)
 Coordinates:
   * latitude   (latitude) float64 1kB 89.5 88.5 87.5 86.5 ... -87.5 -88.5 -89.5
   * longitude  (longitude) float32 1kB 0.0 1.0 2.0 3.0 ... 357.0 358.0 359.0
   * lead_time  (lead_time) timedelta64[ns] 16B 7 days 14 days
   * time       (time) datetime64[ns] 0B ,
 '2t': <xarray.DataArray '2t' (lead_time: 2, latitude: 180, longitude: 360, time: 0)> Size: 0B
 array([], shape=(2, 180, 360, 0), dtype=float32)
 Co

In [37]:
true_trajectory-100

<xarray.DataArray 'q' (level: 3, lead_time: 3, latitude: 180, longitude: 360)> Size: 2MB
array([[[[100181.9  , 100181.44 , 100180.18 , ..., 100182.055,
          100181.836, 100181.61 ],
         [100049.83 , 100056.08 , 100062.414, ..., 100036.625,
          100040.95 , 100045.375],
         [ 99920.89 ,  99923.05 ,  99920.23 , ...,  99903.79 ,
           99907.82 ,  99912.04 ],
         ...,
         [101000.92 , 100994.67 , 100988.336, ..., 101017.17 ,
          101011.76 , 101006.375],
         [100977.984, 100974.12 , 100970.2  , ..., 100988.18 ,
          100984.79 , 100981.4  ],
         [100953.96 , 100952.64 , 100951.29 , ..., 100958.125,
          100956.75 , 100955.305]],

        [[101413.64 , 101413.234, 101411.945, ..., 101412.375,
          101412.61 , 101412.805],
         [101434.266, 101441.875, 101449.46 , ..., 101419.17 ,
          101424.195, 101429.234],
         [101393.234, 101391.18 , 101383.   , ..., 101386.   ,
          101385.96 , 101386.02 ],
...
         [100159.484, 100157.375, 100155.39 , ..., 100166.234,
          100163.805, 100161.57 ],
         [100381.445, 100380.94 , 100380.52 , ..., 100383.695,
          100382.836, 100382.07 ],
         [100605.52 , 100605.58 , 100605.67 , ..., 100606.   ,
          100605.82 , 100605.64 ]],

        [[102239.44 , 102229.35 , 102217.03 , ..., 102265.08 ,
          102256.04 , 102246.84 ],
         [102802.72 , 102805.55 , 102808.3  , ..., 102797.914,
          102799.555, 102801.16 ],
         [102504.61 , 102500.66 , 102496.64 , ..., 102513.09 ,
          102508.96 , 102505.125],
         ...,
         [100869.08 , 100862.82 , 100856.64 , ..., 100883.78 ,
          100878.9  , 100874.06 ],
         [100826.64 , 100824.07 , 100821.58 , ..., 100831.37 ,
          100829.875, 100828.28 ],
         [100768.016, 100767.91 , 100767.75 , ..., 100768.54 ,
          100768.39 , 100768.164]]]], dtype=float32)
Coordinates:
  * level      (level) int64 24B 1000 700 250
  * longitude  (longitude) float64 3kB 0.0 1.0 2.0 3.0 ... 357.0 358.0 359.0
  * latitude   (latitude) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * lead_time  (lead_time) timedelta64[ns] 24B 7 days 14 days 21 days